# Notebook: Train Model

This notebook is used to train a classification model given a dataset of tweets. Results of the training are saved in CSV and JSON.
<br>**Contributors:** [Nils Hellwig](https://github.com/NilsHellwig/) | [Markus Bink](https://github.com/MarkusBink/)

## Packages

In [1]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from simpletransformers.classification import ClassificationModel
from get_germeval_2017_dataset import get_germeval_2017_dataset
from get_schmidt_2022_dataset import get_schmidt_2022_dataset
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import shutil
import random
import json
import os

## Parameters

In [2]:
# PATHS
SPLIT_ID = 0
TEST_DATASET_PATH = f'../Datasets/k_fold_splits/TRAIN_TEST_{SPLIT_ID}/test.csv'
MODEL_NAME = f'BERT-4_{SPLIT_ID}'
MODEL_DIRECTORY_PATH = f'../Trainings/Models/' + MODEL_NAME
PATH_RESULT_DATA = f'../Trainings/Results/' + MODEL_NAME

# HYPERPARAMETERS
N_TRAIN_EPOCHS = 4
TRAIN_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32
USE_CUDA = True

# OTHER MODEL SETTINGS
SAVE_MODEL = False
N_LABELS = 3
EVALUATE_MODEL = True
SEED_VALUE = 0
LABEL_DEFINITION = {'negative': 1, 'positive': 0, 'neutral': 2}

## MODEL TYPE
MODEL_TYPE = "bert"
MODEL_NAME = "deepset/gbert-base"

## Code

### 1. Get Reproducable Results

In [3]:
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)

### 2. Load Dataframes

#### Load Training Data
**Important:** Comment out unnecessary data frames

In [4]:
train_df_annotated_split = pd.read_csv(f'../Datasets/k_fold_splits/TRAIN_TEST_{SPLIT_ID}/train.csv', encoding="utf-8")[["tweet","sentiment"]].rename(columns={"tweet":"text"})
train_df_germeval = get_germeval_2017_dataset()
train_df_schmidt = get_schmidt_2022_dataset()
train_df_annotated_total = pd.read_csv("../Datasets/annotations_matched_filtered.csv", encoding="utf-8")[["tweet","sentiment"]].rename(columns={"tweet":"text"})

In [6]:
train_df = pd.concat([train_df_annotated_split, train_df_germeval, train_df_schmidt], axis=0).sample(frac=1, random_state=SEED_VALUE).reset_index(drop=True)
train_df['sentiment'] = train_df['sentiment'].str.lower()

#### Load Test Data

In [7]:
if EVALUATE_MODEL:
    test_df = pd.read_csv(TEST_DATASET_PATH, encoding="utf-8")[["tweet","sentiment"]].rename(columns={"tweet":"text"})
    test_df['sentiment'] = test_df['sentiment'].str.lower()

#### Replace label strings with numbers

In [8]:
train_df['sentiment'] = train_df['sentiment'].replace(LABEL_DEFINITION)

In [9]:
if EVALUATE_MODEL:
    test_df['sentiment'] = test_df['sentiment'].replace(LABEL_DEFINITION)

### 3. Create Model

In [ ]:
training_args = {
    "fp16":False,
    "num_train_epochs":N_TRAIN_EPOCHS,
    "overwrite_output_dir":True,
    "train_batch_size":TRAIN_BATCH_SIZE,
    "eval_batch_size":TEST_BATCH_SIZE,
    "manual_seed": SEED_VALUE,
    "reprocess_input_data":True,
    "no_cache":True,
    "use_multiprocessing":False,
    "use_multiprocessing_for_evaluation":False,
    "save_model_every_epoch":False
}

In [ ]:
model = ClassificationModel(model_type=MODEL_TYPE, model_name=MODEL_NAME, num_labels=N_LABELS, args=training_args, use_cuda=USE_CUDA)

### 4. Train Model

In [ ]:
model.train_model(train_df)

### 5. Define Metrics

In [ ]:
accuracy_metric = accuracy_score

def f1_metrics(labels, predictions):
    metrics = {
      "f1_macro": f1_score(labels, predictions, average='macro'),
      "f1_micro": f1_score(labels, predictions, average='micro'),
      "f1_weighted": f1_score(labels, predictions, average='weighted')
    }
    return metrics

def precision_metrics(labels, predictions):
    metrics = {
      "precision_macro": precision_score(labels, predictions, average='macro'),
      "precision_micro": precision_score(labels, predictions, average='micro'),
      "precision_weighted": precision_score(labels, predictions, average='weighted')
    }
    return metrics

def recall_metrics(labels, predictions):
    metrics = {
      "recall_macro": recall_score(labels, predictions, average='macro'),
      "recall_micro": recall_score(labels, predictions, average='micro'),
      "recall_weighted": recall_score(labels, predictions, average='weighted')
    }
    return metrics

In [ ]:
def precision_recall_each_class(labels, predictions):
    precision_recall = {}
    for c in set(labels):
        label_idx = [i for i, x in enumerate(labels) if x == c]
        pred_idx = [i for i, x in enumerate(predictions) if x == c]
        precision = len(set(label_idx).intersection(set(pred_idx))) / len(pred_idx) if len(pred_idx) > 0 else 0
        recall = len(set(label_idx).intersection(set(pred_idx))) / len(label_idx) if len(label_idx) > 0 else 0
        precision_recall[c] = {"precision": precision, "recall": recall}
    return precision_recall

### 6. Create Directories for Models and Results

In [ ]:
try:
    os.makedirs("../Trainings")
except FileExistsError:
    # The directory already exists, so do nothing
    pass

try:
    os.makedirs("../Trainings/Results")
except FileExistsError:
    # The directory already exists, so do nothing
    pass

try:
    os.makedirs("../Trainings/Models")
except FileExistsError:
    # The directory already exists, so do nothing
    pass

### 7. Evaluate Model

In [ ]:
if EVALUATE_MODEL:
    result, model_outputs, wrong_predictions = model.eval_model(test_df, acc=accuracy_metric, f1=f1_metrics, precision=precision_metrics, recall=recall_metrics, precision_recall_each_class=precision_recall_each_class)

In [ ]:
if EVALUATE_MODEL:
    with open(PATH_RESULT_DATA+".json", 'w') as f:
        json.dump(result, f, default=str)

### 8. Save Evaluated Test Dataframe

In [ ]:
test_data = test_df
texts = []
for index, row in test_data.iterrows():
    texts.append(row["text"])
predictions, raw_outputs = model.predict(texts)

In [ ]:
test_df_out = test_df.assign(pred=pd.Series(predictions))
test_df_out.to_csv(PATH_RESULT_DATA+".csv")

### 9. Save Model

In [ ]:
def save_model(dest_folder):
    src_folder = "outputs"
    for item in os.listdir(src_folder):
        src_item = os.path.join(src_folder, item)
        dest_item = os.path.join(dest_folder, item)
        if os.path.isdir(src_item):
            copy_folder_contents(src_item, dest_item)
        else:
            shutil.copy(src_item, dest_item)

if SAVE_MODEL:
    save_model(MODEL_DIRECTORY_PATH)